# **Task11**

Use the code from: https://medium.com/@rekalantar/pytorch-tutorial-dynamic-weight-pruning-for-more-optimized-and-faster-neural-networks-7b337e47987b to do dynamic weight pruning. If the full dataset is too large use a suitable subset size.

Use post-training static quantization as well as dynamic quantization as given in: https://pytorch.org/tutorials/recipes/quantization.html to generate a dynamic weight pruning+quantized model. Compare accuracy of the quantized model vs the non-quantized model.


## Importing the required Libraries

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchsummary
from torchvision import models, datasets
from timeit import default_timer as timer
import torch.quantization as quant
import os

Checking for device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Loading the CIFAR10 dataset

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# Load and subset the dataset
full_dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(full_dataset, batch_size=100, shuffle=True)
# Load the test dataset
test_dataset = datasets.CIFAR10('./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

Files already downloaded and verified


# Creating a Model

In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, input_channels, out_planes, stride= 1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, out_planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 =  nn.Conv2d(out_planes, out_planes, kernel_size=3,padding=1)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.downsample = downsample
        # FloatFunction()
        self.skip_add = nn.quantized.FloatFunctional()
        

    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample is not None:
            identity = self.downsample(identity)
        # Notice the addition operation in both scenarios
        x  = self.skip_add.add(x, identity)
        x = self.relu(x)
        
        return x


class ResNet(nn.Module):
    
    def __init__(self, block=BasicBlock, layers=[2, 2, 2, 2], num_channels =3 ,num_classes=10):
        super(ResNet, self).__init__()
        self.input_channels = 64
        self.conv1 = nn.Conv2d(num_channels, self.input_channels, kernel_size = 7, stride = 2, padding = 3)
        self.bn1 = nn.BatchNorm2d(self.input_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.layer1 = self._make_layer(block, layers[0], out=64,  stride= 1)
        self.layer2 = self._make_layer(block, layers[1], out=128, stride= 2)
        self.layer3 = self._make_layer(block, layers[2], out=256, stride= 2)
        self.layer4 = self._make_layer(block, layers[3], out=512, stride= 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def _make_layer(self, block, residual_blocks ,out, stride):
        downsample = None
        layers = []
        if stride != 1 or self.input_channels != out:
            downsample = nn.Sequential(nn.Conv2d(self.input_channels, out, kernel_size=1, stride= stride),
                                       nn.BatchNorm2d(out))
        layers.append(block(self.input_channels, out, stride, downsample))
        self.input_channels = out
        for _ in range(1, residual_blocks):
            layers.append(block(self.input_channels, out))

        return nn.Sequential(*layers)

    
    def forward(self, x):
        # Input are quantized
        x = self.quant(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.dequant(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x

Calling the model ResNet()

In [6]:
model = ResNet()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

### Defining the optimizer and loss function

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### Function to count the zero parameters in the model

In [8]:
def count_zero_params(model):
    zero_count = 0
    for param in model.parameters():
        zero_count += torch.sum(param == 0).item()
    return zero_count

Printing the No.of Zero parameters

In [9]:
before_pruning = count_zero_params(model)
print(before_pruning)

4800


Printing the weights of the model

In [10]:
print("Original weights:\n", model.conv1.weight)

Original weights:
 Parameter containing:
tensor([[[[-0.0524,  0.0089, -0.0540,  ..., -0.0436,  0.0112,  0.0015],
          [ 0.0818,  0.0015, -0.0470,  ...,  0.0530,  0.0092, -0.0446],
          [ 0.0023,  0.0574, -0.0408,  ..., -0.0574,  0.0068, -0.0247],
          ...,
          [-0.0500, -0.0031,  0.0059,  ...,  0.0519,  0.0768,  0.0786],
          [ 0.0148,  0.0448, -0.0210,  ..., -0.0606, -0.0476,  0.0415],
          [ 0.0300, -0.0013,  0.0273,  ..., -0.0186,  0.0618,  0.0060]],

         [[-0.0252,  0.0551,  0.0318,  ..., -0.0363,  0.0460, -0.0005],
          [ 0.0265, -0.0037, -0.0777,  ..., -0.0225,  0.0480,  0.0665],
          [-0.0330, -0.0513,  0.0290,  ..., -0.0266, -0.0374, -0.0117],
          ...,
          [-0.0200,  0.0319,  0.0057,  ...,  0.0118, -0.0097,  0.0103],
          [-0.0616, -0.0073, -0.0560,  ...,  0.0346, -0.0388, -0.0264],
          [-0.0190, -0.0510,  0.0240,  ...,  0.0084, -0.0357, -0.0620]],

         [[-0.0458, -0.0320, -0.0287,  ..., -0.0085, -0.0529,

Function to train the model and to do weight pruning

In [11]:
import torch.nn.utils.prune as prune

def prune_model(model, pruning_rate=0.1):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            
            # Applying unstructured L1 norm pruning
            prune.l1_unstructured(module, name='weight', amount=pruning_rate)
            
            prune.remove(module, 'weight')

In [12]:
def train_pruned_model(model, device, train_loader, optimizer, criterion,epochs,prune_epoch):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss : {running_loss / len(train_loader)}')       
        
        if (epoch+1) % prune_epoch == 0:
            prune_model(model, pruning_rate=0.1)
            print('Pruning done.')

## Calling the function train_model to train 

In [13]:
train_pruned_model(model, device, train_loader, optimizer,criterion, epochs=15,prune_epoch=5)

Epoch [1/15], Loss : 1.4877073529958724
Epoch [2/15], Loss : 1.0292752691507339
Epoch [3/15], Loss : 0.8216779981851577
Epoch [4/15], Loss : 0.6850200694203377
Epoch [5/15], Loss : 0.5741130374073983
Pruning done.
Epoch [6/15], Loss : 0.48218989551067354
Epoch [7/15], Loss : 0.4089094673991203
Epoch [8/15], Loss : 0.33029090958833696
Epoch [9/15], Loss : 0.27293209539353847
Epoch [10/15], Loss : 0.220408936470747
Pruning done.
Epoch [11/15], Loss : 0.1782695506811142
Epoch [12/15], Loss : 0.14709078270941972
Epoch [13/15], Loss : 0.12446953675895929
Epoch [14/15], Loss : 0.10481965002603828
Epoch [15/15], Loss : 0.08701379926875234
Pruning done.


Printing the weights after weight pruning

In [14]:
print("Pruned weights:\n", model.conv1.weight)

Pruned weights:
 Parameter containing:
tensor([[[[ 0.0260,  0.0481, -0.0000,  ...,  0.0304,  0.0649, -0.0310],
          [ 0.0676, -0.0616, -0.1236,  ...,  0.0000, -0.0000, -0.0704],
          [-0.0378, -0.0186, -0.1223,  ..., -0.1802, -0.0868, -0.1080],
          ...,
          [ 0.0000,  0.0372,  0.0431,  ...,  0.1583,  0.1463,  0.1324],
          [ 0.0388,  0.0568, -0.0159,  ..., -0.0628, -0.0247,  0.0224],
          [ 0.1341,  0.1095,  0.1253,  ...,  0.0631,  0.1341,  0.0000]],

         [[ 0.1054,  0.1132,  0.1097,  ...,  0.0785,  0.1831,  0.0746],
          [ 0.0205, -0.0969, -0.1763,  ..., -0.0875,  0.0521,  0.0853],
          [-0.0811, -0.1372, -0.0440,  ..., -0.1647, -0.1625, -0.1277],
          ...,
          [-0.0765, -0.0339, -0.0243,  ...,  0.0388, -0.0726, -0.0414],
          [-0.1704, -0.1358, -0.1861,  ..., -0.1241, -0.1798, -0.1472],
          [-0.0602, -0.1049, -0.0178,  ..., -0.0941, -0.1335, -0.1664]],

         [[-0.0233, -0.0928, -0.0514,  ..., -0.0000, -0.0162, -

No.of Zero parameters after weight pruning

In [15]:
after_pruning = count_zero_params(model)
print(after_pruning)

1117203


Function to Evaluate the model

In [16]:
import time
def evaluate_model(model,device):
    model = model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            st = time.time()
            outputs = model(images)
            et = time.time()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct // total} %')
    print('Elapsed time = {:0.4f} milliseconds'.format((et - st) * 1000))
    print("====================================================================================================")

Evaluating the model and printing the accuracy

In [17]:
device ='cpu'
print("=====================================AFTER WEIGHT PRUNING========================================")
evaluate_model(model,device)

=====================================AFTER WEIGHT PRUNING========================================
Accuracy: 75 %
Elapsed time = 62.7477 milliseconds


Function to print the size of the model

In [18]:
def print_size_of_model(model,path):
    torch.save(model.state_dict(), path)
    print('Size (MB):', os.path.getsize(path)/1e6)

Printing the size of the model before quantization 

In [19]:
print("=====================================BEFORE QUANTIZATION========================================")
print(model)
print('Size of the model Before Dynamic quantization')
print_size_of_model(model,path="FinalTask11.pt")

=====================================BEFORE QUANTIZATION========================================
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddin

# Dynamic quantization

Doing dynamic quantization for the model

In [20]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)

Printing the model and size of it after dynamic quantization

In [21]:
print("=====================================DYNAMIC QUANTIZATION========================================")
print(model_dynamic_quantized)
print('Size of the model After Dynamic quantization')
print_size_of_model(model_dynamic_quantized,path="FinalTask11_dynamic.pt")

=====================================DYNAMIC QUANTIZATION========================================
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddi

# Static Quantization

Doing Static quantization for the model

In [22]:
model_dynamic_quantized.eval()
model_dynamic_quantized.qconfig = torch.quantization.get_default_qconfig('fbgemm')

In [23]:
static_quantized = torch.quantization.prepare(model_dynamic_quantized, inplace=True)
print(static_quantized)

ResNet(
  (conv1): Conv2d(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)
    (activation_post_process): HistogramObserver()
  )
  (bn1): BatchNorm2d(
    64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (activation_post_process): HistogramObserver()
  )
  (relu): ReLU(
    inplace=True
    (activation_post_process): HistogramObserver()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        (activation_post_process): HistogramObserver()
      )
      (bn1): BatchNorm2d(
        64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (activation_post_process): HistogramObserver()
      )
      (relu): ReLU(
        inplace=True
        (activation_post_process): HistogramObserver()
      )
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1

In [31]:
with torch.no_grad():
    for data, target in train_loader:
        static_quantized(data)

In [27]:
static_quantized = torch.quantization.convert(static_quantized, inplace=True)
print(static_quantized)

ResNet(
  (conv1): QuantizedConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.0864262655377388, zero_point=63, padding=(3, 3))
  (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): QuantizedReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.25623172521591187, zero_point=68, padding=(1, 1))
      (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): QuantizedReLU(inplace=True)
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.10178610682487488, zero_point=71, padding=(1, 1))
      (bn2): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): QFunctional(
        scale=0.13635751605033875, zero_point=42
       

In [32]:
torch.jit.save(torch.jit.script(static_quantized), "static_.pth")

In [33]:
def print_size_of_quantized_model(path):
    print('Size (MB):', os.path.getsize(path)/1e6)

Printing the model and size of it after Static quantization

In [34]:
print('Size of the model After Static quantization')
print_size_of_quantized_model(path="static_.pth")

Size of the model After Static quantization
Size (MB): 11.429824


Evaluating and finding the accuracy of the model 

In [35]:
device ='cpu'
print("=====================================ACCURACY AFTER STATIC QUANTIZATION=======================================")
evaluate_model(static_quantized,device)

=====================================ACCURACY AFTER STATIC QUANTIZATION=======================================
Accuracy: 75 %
Elapsed time = 10.3590 milliseconds
